In [ ]:
!pip install opencv-python opencv-python-headless

In [ ]:
import cv2
import os

# Set the video capture properties
camera_index = 0  # Change this if you have multiple cameras
frame_width = 640
frame_height = 480
fps = 30
video_format = 'XVID'
video_file_path = '/Users/thinnaphatd/Dev_work/deep_learning/Image/video2.mp4'

# Create the output directory if it doesn't exist
output_directory = os.path.dirname(video_file_path)
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Initialize the camera
cap = cv2.VideoCapture(camera_index)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, frame_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_height)
cap.set(cv2.CAP_PROP_FPS, fps)

# Initialize the video writer
fourcc = cv2.VideoWriter_fourcc(*video_format)
out = cv2.VideoWriter(video_file_path, fourcc, fps, (frame_width, frame_height))

print("Press 'q' to stop recording and save the video.")

while cap.isOpened():
    # Capture a frame from the camera
    ret, frame = cap.read()

    if ret:
        # Write the frame to the output file
        out.write(frame)

        # Display the frame
        cv2.imshow('Camera', frame)

        # Check if 'q' key is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# Release the camera and video writer, and close the display window
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Video saved to {video_file_path}.")


In [ ]:
import cv2

def capture_roi(video_file_path, output_image_path, roi_coordinates, frame_number):
    cap = cv2.VideoCapture(video_file_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    if frame_number >= frame_count:
        print(f"Frame number {frame_number} is out of range. The video has {frame_count} frames.")
        return

    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    ret, frame = cap.read()

    if ret:
        x, y, w, h = roi_coordinates
        roi = frame[y:y+h, x:x+w]
        
        cv2.imwrite(output_image_path, roi)
        print(f"Saved ROI image to {output_image_path}")
    else:
        print(f"Error reading frame {frame_number}")

    cap.release()

video_file_path = '/Users/thinnaphatd/Dev_work/deep_learning/Image/video2.mp4'  # Change this to your video file path
output_image_path = '/Users/thinnaphatd/Dev_work/deep_learning/Image/roi_image5.jpg'
roi_coordinates = (0, 0, 640, 480)  # Replace these with the coordinates (x, y) and dimensions (w, h) of the desired ROI
frame_number = 527  # Change this to the desired frame number

capture_roi(video_file_path, output_image_path, roi_coordinates, frame_number)


In [ ]:
!pip install tensorflow


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import cv2

# Match contours to license plate or character template
def find_contours(dimensions, img) :

    # Find all contours in the image
    cntrs, _ = cv2.findContours(img.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Retrieve potential dimensions
    lower_width = dimensions[0]
    upper_width = dimensions[1]
    lower_height = dimensions[2]
    upper_height = dimensions[3]
    
    # Check largest 5 or  15 contours for license plate or character respectively
    cntrs = sorted(cntrs, key=cv2.contourArea, reverse=True)[:15]
    
    ii = cv2.imread('/Users/thinnaphatd/Dev_work/deep_learning/Image/roi_image5.jpg')
    
    
    x_cntr_list = []
    target_contours = []
    img_res = []
    for cntr in cntrs :
        #detects contour in binary image and returns the coordinates of rectangle enclosing it
        intX, intY, intWidth, intHeight = cv2.boundingRect(cntr)
        
        #checking the dimensions of the contour to filter out the characters by contour's size
        if intWidth > lower_width and intWidth < upper_width and intHeight > lower_height and intHeight < upper_height :
            x_cntr_list.append(intX) #stores the x coordinate of the character's contour, to used later for indexing the contours

            char_copy = np.zeros((44,24))
            #extracting each character using the enclosing rectangle's coordinates.
            char = img[intY:intY+intHeight, intX:intX+intWidth]
            char = cv2.resize(char, (20, 40))
            
            cv2.rectangle(ii, (intX,intY), (intWidth+intX, intY+intHeight), (50,21,200), 2)
            plt.imshow(ii, cmap='gray')

#             Make result formatted for classification: invert colors
            char = cv2.subtract(255, char)

            # Resize the image to 24x44 with black border
            char_copy[2:42, 2:22] = char
            char_copy[0:2, :] = 0
            char_copy[:, 0:2] = 0
            char_copy[42:44, :] = 0
            char_copy[:, 22:24] = 0

            img_res.append(char_copy) #List that stores the character's binary image (unsorted)
            
    #Return characters on ascending order with respect to the x-coordinate (most-left character first)
            
    plt.show()
    #arbitrary function that stores sorted list of character indeces
    indices = sorted(range(len(x_cntr_list)), key=lambda k: x_cntr_list[k])
    img_res_copy = []
    for idx in indices:
        img_res_copy.append(img_res[idx])# stores character images according to their index
    img_res = np.array(img_res_copy)

    return img_res

In [ ]:
# Find characters in the resulting images
def segment_characters(image) :

    # Preprocess cropped license plate image
    img_lp = cv2.resize(image, (300, 75))
    img_gray_lp = cv2.cvtColor(img_lp, cv2.COLOR_BGR2GRAY)
    _, img_binary_lp = cv2.threshold(img_gray_lp, 200, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    img_binary_lp = cv2.erode(img_binary_lp, (3,3))
    img_binary_lp = cv2.dilate(img_binary_lp, (3,3))

    LP_WIDTH = img_binary_lp.shape[0]
    LP_HEIGHT = img_binary_lp.shape[1]

    # Make borders white
    img_binary_lp[0:3,:] = 255
    img_binary_lp[:,0:3] = 255
    img_binary_lp[72:75,:] = 255
    img_binary_lp[:,330:333] = 255

    # Estimations of character contours sizes of cropped license plates
    dimensions = [LP_WIDTH/6,
                       LP_WIDTH/2,
                       LP_HEIGHT/10,
                       2*LP_HEIGHT/3]
    plt.imshow(img_binary_lp, cmap='gray')
    plt.show()
    cv2.imwrite('contour.jpg',img_binary_lp)

    # Get contours within cropped license plate
    char_list = find_contours(dimensions, img_binary_lp)

    return char_list

In [ ]:
img = cv2.imread('/Users/thinnaphatd/Dev_work/deep_learning/Image/roi_image5.jpg')
char = segment_characters(img)

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the pre-trained Keras model
model = load_model('/Users/thinnaphatd/Dev_work/deep_learning/ENGcharacter_weightFinal.h5')

# Define the video file path
video_file_path = '/Users/thinnaphatd/Dev_work/deep_learning/Image/video1.avi'

# Initialize the video capture
cap = cv2.VideoCapture(video_file_path)

# Define the preprocessing function (depending on your model)
def preprocess_frame(frame):
    # Resize the frame, normalize the pixel values, and expand the dimensions
    frame_resized = cv2.resize(frame, (28, 28))  # Change the size according to your model's input
    frame_normalized = frame_resized.astype('float32') / 255.0
    frame_expanded = np.expand_dims(frame_normalized, axis=0)
    return frame_expanded

# Process the video frames and make predictions
while cap.isOpened():
    ret, frame = cap.read()

    if ret:
        def fix_dimension(img): 
             new_img = np.zeros((28,28,3))
             for i in range(3):
                  new_img[:,:,i] = img
             return new_img
        # Preprocess the frame
        processed_frame = preprocess_frame(frame)

        # Make a prediction using the model
        # prediction = model.predict(processed_frame)

        # Process the prediction (e.g., print the predicted class)
        def show_results():
            dic = {}
            characters = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ'
            for i,c in enumerate(characters):
                  dic[i] = c
            output = []
            for i,ch in enumerate(char): #iterating over the characters
                img_ = cv2.resize(ch, (28,28), interpolation=cv2.INTER_AREA)
                img = fix_dimension(img_)
                img = img.reshape(1,28,28,3) #preparing image for the model
                y_ = model.predict(img)[0] #predicting the class
                for num in range(len(y_)):
                    if y_[num] == 1:
                        y_num = num
#         print(img)
#         print(y_)
#         print(dic)
        
                character = dic[y_num] 
#         print(character)
                output.append(character) #storing the result in a list
        
        
        
            plate_number = ''.join(output)
    
            return plate_number

        print(show_results())

        # predicted_class = np.argmax(prediction)
        # print(f"Predicted class: {predicted_class}")

#         # Display the frame
#         cv2.imshow('Video', frame)

#         # Check if 'q' key is pressed
#         if cv2.waitKey(25) & 0xFF == ord('q'):
#             break
#     else:
#         break

# # Release the video capture and close the display window
# cap.release()
# cv2.destroyAllWindows()


In [8]:
import cv2

# Load the pre-trained model
model_weights = "/Users/thinnaphatd/Dev_work/deep_learning/resnet10_fd_lpd.caffemodel"
model_config = "/Users/thinnaphatd/Dev_work/deep_learning/resnet10_fd_lpd.prototxt"
net = cv2.dnn.readNetFromCaffe(model_config, model_weights)

# Set up camera
cap = cv2.VideoCapture(0)

while True:
    # Read the current frame
    ret, frame = cap.read()
    if not ret:
        break

    # Prepare the frame for object detection
    h, w = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (104.0, 177.0, 123.0))

    # Perform object detection
    net.setInput(blob)
    detections = net.forward()

    # Process detections
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:  # Set the confidence threshold
            class_id = int(detections[0, 0, i, 1])

            # Check if detected object is a license plate
            if class_id == 1:
                box = detections[0, 0, i, 3:7] * [w, h, w, h]
                (startX, startY, endX, endY) = box.astype("int")
                cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)

    # Show the result
    cv2.imshow("License Plate Detection", frame)

    # Exit the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [ ]:
import cv2

# Load the pre-trained model
model_weights = "/Users/thinnaphatd/Dev_work/deep_learning/resnet10_fd_lpd.caffemodel"
model_config = "/Users/thinnaphatd/Dev_work/deep_learning/resnet10_fd_lpd.prototxt"
net = cv2.dnn.readNetFromCaffe(model_config, model_weights)

# Set up camera
cap = cv2.VideoCapture(0)

while True:
    # Read the current frame
    ret, frame = cap.read()
    if not ret:
        break

    # Prepare the frame for object detection
    h, w = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (104.0, 177.0, 123.0))

    # Perform object detection
    net.setInput(blob)
    detections = net.forward()

    # Process detections
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.2:  # Set the confidence threshold
            class_id = int(detections[0, 0, i, 1])

            # Check if detected object is a license plate
            if class_id == 1:
                box = detections[0, 0, i, 3:7] * [w, h, w, h]
                (startX, startY, endX, endY) = box.astype("int")
                cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)

    # Show the result
    cv2.imshow("License Plate Detection", frame)

    # Exit the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()